由于vLLM默认并不支持 CPU 部署和推理，为了验证和演示CPU部署和推理过程,同时也因vLLM通过CPU部署推理的文档少之又少，官网有些地方也不是很明确，导致在执行过程中走了不少弯路。
1. CPU模式下，如果编译打包vLLM框架（你没有看错：CPU模式需要我们自己编译打包）
2. 通过CPU模式，演示多种方式部署和推理Qwen2大模型（其他模型也一样）：本地模式、部署 API 服务、API 客户端、WebUI 等方式
3. 最后，在GPU模式，简单介绍vLLM部署和推理；相比CPU环境，简单太多了，因为vLLM默认就只支持GPU环境

vLLM 官网源代码地址：https://github.com/vllm-project/vllm

vLLM 支持目前主流大模型，详细列表见官网：https://docs.vllm.ai/en/latest/models/supported_models.html

vllm cpu版本安装教程：https://docs.vllm.ai/en/latest/getting_started/cpu-installation.html

特别注意的是：vLLM目前只支持Linux操作系统（包括Windows WSL子系统）：如果是Linux系统，那就无需其他操作；如果是Windows操作系统，需要首先安装和配置好WSL子系统。

## 使用 vLLM 部署和推理大模型

特别注意：vLLM的依赖包默认支持 GPU 部署和推理，如果使用CPU推理，我们需要根据vLLM源代码重新编译打包！

【GPU 部署和推理】比较简单，通过 PIP 直接安装依赖包即可：

In [ ]:
pip install vLLM

【CPU 部署和推理】我们需要下载vLLM源代码，自己编译打包和安装：

In [ ]:
# 编译GCC
sudo apt-get update  -y
sudo apt-get install -y gcc-12 g++-12 libnuma-dev
sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-12 10 --slave /usr/bin/g++ g++ /usr/bin/g++-12

In [ ]:
# 下载vLLM代码
#!git clone https://github.com/vllm-project/vllm.git vllm-project
#%cd vllm-project

# 下载稳定版本
git clone https://github.com/vllm-project/vllm.git vllm-project -b v0.6.2
%cd vllm-project

In [ ]:
# 安装vLLM便于打包的依赖
pip install --upgrade pip

pip install wheel packaging ninja "setuptools>=49.4.0" numpy

pip install -v -r requirements-cpu.txt --extra-index-url https://download.pytorch.org/whl/cpu

In [ ]:
# https://blog.csdn.net/weixin_41010198/article/details/109343347
# 添加软路由
sudo ln -s /root/cmake-3.27.0-linux-x86_64/bin/cmake /usr/bin/cmake
# 删除软路由
sudo rm /usr/bin/cmake
# https://blog.csdn.net/qq_42951560/article/details/121438924

In [ ]:
!git clone -b rls-v3.5 https://github.com/oneapi-src/oneDNN.git

!cmake -B ./oneDNN/build -S ./oneDNN -G Ninja -DONEDNN_LIBRARY_TYPE=STATIC \
    -DONEDNN_BUILD_DOC=OFF \
    -DONEDNN_BUILD_EXAMPLES=OFF \
    -DONEDNN_BUILD_TESTS=OFF \
    -DONEDNN_BUILD_GRAPH=OFF \
    -DONEDNN_ENABLE_WORKLOAD=INFERENCE \
    -DONEDNN_ENABLE_PRIMITIVE=MATMUL

!cmake --build ./oneDNN/build --target install --config Release

In [ ]:
# vLLM打包安装
VLLM_TARGET_DEVICE=cpu python setup.py install

## vLLM 本地大模型部署和推理测试

In [ ]:
# Qwen2-vLLM-Local.py
import os
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams
 
# 设置环境变量
os.environ['VLLM_TARGET_DEVICE'] = 'cpu'
 
# 模型ID：我们下载的模型权重文件目录
model_dir = 'AI-ModelScope/Qwen2___5-7B-Instruct'
 
# Tokenizer初始化
tokenizer = AutoTokenizer.from_pretrained(
    model_dir,
    local_files_only=True,
)
 
# Prompt提示词
messages = [
    {'role': 'system', 'content': 'You are a helpful assistant.'},
    {'role': 'user', 'content': '天空为什么是蓝色的？'}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
 
# 初始化大语言模型
llm = LLM(
    model=model_dir,
    tensor_parallel_size=1,  # CPU无需张量并行
    device='cpu',
)

# 超参数：最多512个Token
sampling_params = SamplingParams(temperature=0.7, top_p=0.8, repetition_penalty=1.05, max_tokens=512)
 
# 模型推理输出
outputs = llm.generate([text], sampling_params)
 
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
 
    print(f'Prompt提示词: {prompt!r}, 大模型推理输出: {generated_text!r}')

In [ ]:
# default_dump_dir报错
# 解决pip install -U --index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/Triton-Nightly/pypi/simple/ triton-nightly

In [ ]:
# 查看物理CPU个数
cat /proc/cpuinfo| grep "physical id"| sort| uniq| wc -l

# 查看每个物理CPU中core的个数(即核数)
cat /proc/cpuinfo| grep "cpu cores"| uniq

# 查看逻辑CPU的个数
cat /proc/cpuinfo| grep "processor"| wc -l

## 发布 API 服务和调用推理

In [ ]:
python -m vllm.entrypoints.openai.api_server --model AI-ModelScope/Qwen2___5-7B-Instruct

In [ ]:
# Qwen2-vLLM-CURL.py
curl http://localhost:8000/v1/chat/completions -H "Content-Type: application/json" -d '{
  "model": "AI-ModelScope/Qwen2___5-7B-Instruct",
  "messages": [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "天空为什么是蓝色的？"}
  ],
  "temperature": 0.7,
  "top_p": 0.8,
  "repetition_penalty": 1.05,
  "max_tokens": 512
}'

In [1]:
# Qwen2-vLLM-OpenAI.py
from openai import OpenAI
 
# OpenAI初始化
client = OpenAI(
    api_key='EMPTY',
    base_url='http://localhost:8000/v1',
)
 
chat_response = client.chat.completions.create(
    model='AI-ModelScope/Qwen2___5-7B-Instruct',
    messages=[
        {'role': 'system', 'content': 'You are a helpful assistant.'},
        {'role': 'user', 'content': '天空为什么是蓝色的？'},
    ],
    temperature=0.7,
    top_p=0.8,
    max_tokens=512,
)
 
print('Qwen2推理结果:', chat_response)

Qwen2推理结果: ChatCompletion(id='chat-7c8961f5b00e4162ab3a8ab1b23456fe', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='天空呈现蓝色的原因是大气中的气体分子和其他小颗粒会散射太阳光。太阳发出的光实际上是一种包含了所有颜色的白光，当这种白光穿过地球大气层时，波长较短的蓝光和紫光比波长较长的红光更容易被大气中的气体分子散射。\n\n然而，人眼对蓝光和紫光的敏感度较低，而且太阳光中的蓝光被散射得更多，使得我们在白天看到的天空呈现蓝色。日落时，太阳光需要穿过更厚的大气层，蓝光和紫光被进一步散射掉，所以我们看到的天空呈现出橙色或红色。', refusal=None, role='assistant', function_call=None, tool_calls=[]), stop_reason=None)], created=1727919840, model='AI-ModelScope/Qwen2___5-7B-Instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=139, prompt_tokens=25, total_tokens=164, completion_tokens_details=None, prompt_tokens_details=None), prompt_logprobs=None)
